In [92]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
import rasterio
from rasterio import plot
from rasterio.plot import show
from rasterio.windows import Window
from rasterio.windows import from_bounds
from rasterio.enums import Resampling
import requests
import json

In [104]:
postcode, streetname, house_number = input("Enter in the following order: postcode,streetname,housenumber ").split()

Enter in the following order: postcode,streetname,housenumber 9340 Spoorstraat 45


In [105]:
print("Postcode: ", postcode)
print("Streetname: ", streetname)
print("House number: ", house_number)

Postcode:  9340
Streetname:  Spoorstraat
House number:  45


In [106]:
r = requests.get(
    "https://api.basisregisters.vlaanderen.be/v1/adresmatch", params={"postcode":int(postcode), "straatnaam":streetname, "huisnummer":int(house_number) }
)
house_info_nested = json.loads(r.content)
house_info_nested

{'adresMatches': [{'identificator': {'id': 'https://data.vlaanderen.be/id/adres/2396989',
    'naamruimte': 'https://data.vlaanderen.be/id/adres',
    'objectId': '2396989',
    'versieId': '2011-04-29T15:22:40+02:00'},
   'detail': 'https://api.basisregisters.vlaanderen.be/v1/adressen/2396989',
   'gemeente': {'objectId': '41034',
    'detail': 'https://api.basisregisters.vlaanderen.be/v1/gemeenten/41034',
    'gemeentenaam': {'geografischeNaam': {'spelling': 'Lede', 'taal': 'nl'}}},
   'postinfo': {'objectId': '9340',
    'detail': 'https://api.basisregisters.vlaanderen.be/v1/postinfo/9340'},
   'straatnaam': {'objectId': '63343',
    'detail': 'https://api.basisregisters.vlaanderen.be/v1/straatnamen/63343',
    'straatnaam': {'geografischeNaam': {'spelling': 'Spoorstraat',
      'taal': 'nl'}}},
   'huisnummer': '45',
   'volledigAdres': {'geografischeNaam': {'spelling': 'Spoorstraat 45, 9340 Lede',
     'taal': 'nl'}},
   'adresPositie': {'point': {'coordinates': [123581.5, 184188.

In [107]:
type(house_info_nested)

dict

In [108]:
house_info = pd.json_normalize(house_info_nested)

In [109]:
df = pd.json_normalize(house_info.adresMatches[0][0])
df

,detail,huisnummer,positieSpecificatie,positieGeometrieMethode,adresStatus,officieelToegekend,adresseerbareObjecten,score,identificator.id,identificator.naamruimte,...,postinfo.objectId,postinfo.detail,straatnaam.objectId,straatnaam.detail,straatnaam.straatnaam.geografischeNaam.spelling,straatnaam.straatnaam.geografischeNaam.taal,volledigAdres.geografischeNaam.spelling,volledigAdres.geografischeNaam.taal,adresPositie.point.coordinates,adresPositie.point.type
0,https://api.basisregisters.vlaanderen.be/v1/ad...,45,gebouweenheid,afgeleidVanObject,inGebruik,True,"[{'objectType': 'gebouweenheid', 'objectId': '...",100.0,https://data.vlaanderen.be/id/adres/2396989,https://data.vlaanderen.be/id/adres,...,9340,https://api.basisregisters.vlaanderen.be/v1/po...,63343,https://api.basisregisters.vlaanderen.be/v1/st...,Spoorstraat,nl,"Spoorstraat 45, 9340 Lede",nl,"[123581.5, 184188.45]",Point


In [110]:
dff = pd.json_normalize(df.adresseerbareObjecten[0][0])
dff

,objectType,objectId,detail
0,gebouweenheid,9044795,https://api.basisregisters.vlaanderen.be/v1/ge...


In [111]:
dff.detail[0]

'https://api.basisregisters.vlaanderen.be/v1/gebouweenheden/9044795'

In [112]:
r = requests.get(dff.detail[0])
house_gebouweenheden_nested = json.loads(r.content)
print(house_gebouweenheden_nested)

{'identificator': {'id': 'https://data.vlaanderen.be/id/gebouweenheid/9044795', 'naamruimte': 'https://data.vlaanderen.be/id/gebouweenheid', 'objectId': '9044795', 'versieId': '2011-04-29T15:22:40+02:00'}, 'geometriePunt': {'point': {'coordinates': [123581.5, 184188.45], 'type': 'Point'}}, 'positieGeometrieMethode': 'afgeleidVanObject', 'gebouweenheidStatus': 'gerealiseerd', 'functie': 'nietGekend', 'gebouw': {'objectId': '9043815', 'detail': 'https://api.basisregisters.vlaanderen.be/v1/gebouwen/9043815'}, 'adressen': [{'objectId': '2396989', 'detail': 'https://api.basisregisters.vlaanderen.be/v1/adressen/2396989'}]}


In [113]:
type(house_gebouweenheden_nested)

dict

In [114]:
house_gebouweenheden = pd.json_normalize(house_gebouweenheden_nested)
house_gebouweenheden

,positieGeometrieMethode,gebouweenheidStatus,functie,adressen,identificator.id,identificator.naamruimte,identificator.objectId,identificator.versieId,geometriePunt.point.coordinates,geometriePunt.point.type,gebouw.objectId,gebouw.detail
0,afgeleidVanObject,gerealiseerd,nietGekend,"[{'objectId': '2396989', 'detail': 'https://ap...",https://data.vlaanderen.be/id/gebouweenheid/90...,https://data.vlaanderen.be/id/gebouweenheid,9044795,2011-04-29T15:22:40+02:00,"[123581.5, 184188.45]",Point,9043815,https://api.basisregisters.vlaanderen.be/v1/ge...


In [115]:
url_gebouwen = house_gebouweenheden['gebouw.detail'][0]

In [116]:
r = requests.get(url_gebouwen)
house_gebouw_nested = json.loads(r.content)
house_gebouw_nested

{'identificator': {'id': 'https://data.vlaanderen.be/id/gebouw/9043815',
  'naamruimte': 'https://data.vlaanderen.be/id/gebouw',
  'objectId': '9043815',
  'versieId': '2011-04-29T13:11:28+02:00'},
 'geometriePolygoon': {'polygon': {'coordinates': [[[123577.47845698893,
      184179.58982590958],
     [123579.22296898812, 184181.67456191406],
     [123582.27551299334, 184179.19212991],
     [123589.99429699779, 184188.29062591866],
     [123587.06565699726, 184190.85792192072],
     [123589.83660099655, 184194.16416192055],
     [123583.58226499707, 184199.39782592654],
     [123571.26636098325, 184184.78240191564],
     [123577.47845698893, 184179.58982590958]]],
   'type': 'Polygon'}},
 'geometrieMethode': 'ingemetenGRB',
 'gebouwStatus': 'gerealiseerd',
 'gebouweenheden': [{'objectId': '9044795',
   'detail': 'https://api.basisregisters.vlaanderen.be/v1/gebouweenheden/9044795'}],
 'percelen': [{'objectId': '41034B0156-00B002',
   'detail': 'https://api.basisregisters.vlaanderen.be/v

In [117]:
house_gebouw = pd.json_normalize(house_gebouw_nested)
house_gebouw

,geometrieMethode,gebouwStatus,gebouweenheden,percelen,identificator.id,identificator.naamruimte,identificator.objectId,identificator.versieId,geometriePolygoon.polygon.coordinates,geometriePolygoon.polygon.type
0,ingemetenGRB,gerealiseerd,"[{'objectId': '9044795', 'detail': 'https://ap...","[{'objectId': '41034B0156-00B002', 'detail': '...",https://data.vlaanderen.be/id/gebouw/9043815,https://data.vlaanderen.be/id/gebouw,9043815,2011-04-29T13:11:28+02:00,"[[[123577.47845698893, 184179.58982590958], [1...",Polygon


In [118]:
house_polygon = house_gebouw['geometriePolygoon.polygon.coordinates'][0]

In [119]:
house_polygon

[[[123577.47845698893, 184179.58982590958],
  [123579.22296898812, 184181.67456191406],
  [123582.27551299334, 184179.19212991],
  [123589.99429699779, 184188.29062591866],
  [123587.06565699726, 184190.85792192072],
  [123589.83660099655, 184194.16416192055],
  [123583.58226499707, 184199.39782592654],
  [123571.26636098325, 184184.78240191564],
  [123577.47845698893, 184179.58982590958]]]